In [1]:
import pandas as pd

def load_lcqmc():
    '''LCQMC文本匹配数据集
    '''
    train = pd.read_csv('https://mirror.coggle.club/dataset/LCQMC.train.data.zip', 
            sep='\t', names=['query1', 'query2', 'label'])

    valid = pd.read_csv('https://mirror.coggle.club/dataset/LCQMC.valid.data.zip', 
            sep='\t', names=['query1', 'query2', 'label'])

    test = pd.read_csv('https://mirror.coggle.club/dataset/LCQMC.test.data.zip', 
            sep='\t', names=['query1', 'query2', 'label'])

    return train, valid, test

In [2]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [3]:
train,valid,test = load_lcqmc()

In [27]:
test

,query1,query2,label
0,谁有狂三这张高清的,这张高清图，谁有,0
1,英雄联盟什么英雄最好,英雄联盟最好英雄是什么,1
2,这是什么意思，被蹭网吗,我也是醉了，这是什么意思,0
3,现在有什么动画片好看呢？,现在有什么好看的动画片吗？,1
4,请问晶达电子厂现在的工资待遇怎么样要求有哪些,三星电子厂工资待遇怎么样啊,0
...,...,...,...
12495,微店怎么开？怎么做代理？,微店怎样代理,1
12496,小学科学三年级上,小学三年级科学,0
12497,冬眠是什么意思？,冬眠的意思是什么,1
12498,天猫有假货吗,天猫卖假货吗,0


In [4]:
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
import pandas as pd
import random
import re

In [10]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encoding = tokenizer(list(train["query1"]), list(train["query2"]), 
                           truncation=True, padding=True, max_length=100)
valid_encoding = tokenizer(list(valid["query1"]), list(valid["query2"]), 
                          truncation=True, padding=True, max_length=100)
test_encoding = tokenizer(list(test["query1"]), list(test["query2"]), 
                          truncation=True, padding=True, max_length=100)

G:\anaconda\envs\pytorch\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\muma\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [22]:
class BertDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['label'] = torch.tensor(int(self.labels[idx]))
        return item
    
    def __len__(self):
        return len(self.encodings['input_ids'])

In [28]:
train_dataset = BertDataset(train_encoding, list(train["label"]))
valid_dataset = BertDataset(valid_encoding, list(valid["label"]))
test_dataset = BertDataset(test_encoding, list(test["label"]))

In [29]:
# 精度计算
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [31]:
from transformers import BertForNextSentencePrediction, AdamW, get_linear_schedule_with_warmup
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)
optim = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * 1
scheduler = get_linear_schedule_with_warmup(optim, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
G:\anaconda\envs\pytorch\lib\site-packages\transformers\optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed 

In [37]:
# 训练函数
def train():
    model.train()
    total_train_loss = 0
    iter_num = 0
    total_iter = len(train_loader)
    best_accuracy = 0
    for batch in tqdm(train_loader):
        # 正向传播
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        total_train_loss += loss.item()
        
        # 反向梯度信息
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # 参数更新
        optim.step()
        scheduler.step()

        iter_num += 1
        if(iter_num % 100==0):
            print("epoth: %d, iter_num: %d, loss: %.4f, %.2f%%" % (epoch, iter_num, loss.item(), iter_num/total_iter*100))
        if(iter_num % 2000==0):
            accuracy = validation()
            if accuracy > best_accuracy:
                model.save_pretrained('G:\\deep_learning\\Coggle\\202301\\models\\bert\\'+'best_model')
                best_accuracy = accuracy
            model.train()
        
    print("Epoch: %d, Average training loss: %.4f"%(epoch, total_train_loss/len(train_loader)))

def validation():
    print("start validating...")
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    for batch in tqdm(valid_dataloader):
        with torch.no_grad():
            # 正常传播
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
        loss = outputs[0]
        logits = outputs[1]

        total_eval_loss += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        
    avg_val_accuracy = total_eval_accuracy / len(valid_dataloader)
    print("Accuracy: %.4f" % (avg_val_accuracy))
    print("Average testing loss: %.4f"%(total_eval_loss/len(valid_dataloader)))
    print("-------------------------------")
    return avg_val_accuracy
    
for epoch in range(2):
    print("------------Epoch: %d ----------------" % epoch)
    train()

------------Epoch: 0 ----------------


  0%|          | 0/14923 [00:00<?, ?it/s]

epoth: 0, iter_num: 100, loss: 0.4770, 0.67%
epoth: 0, iter_num: 200, loss: 0.3446, 1.34%
epoth: 0, iter_num: 300, loss: 0.5103, 2.01%
epoth: 0, iter_num: 400, loss: 0.5488, 2.68%
epoth: 0, iter_num: 500, loss: 0.2589, 3.35%
epoth: 0, iter_num: 600, loss: 0.3024, 4.02%
epoth: 0, iter_num: 700, loss: 0.4811, 4.69%
epoth: 0, iter_num: 800, loss: 0.2117, 5.36%
epoth: 0, iter_num: 900, loss: 0.3745, 6.03%
epoth: 0, iter_num: 1000, loss: 0.7474, 6.70%
epoth: 0, iter_num: 1100, loss: 0.3683, 7.37%
epoth: 0, iter_num: 1200, loss: 0.4584, 8.04%
epoth: 0, iter_num: 1300, loss: 0.6007, 8.71%
epoth: 0, iter_num: 1400, loss: 0.4657, 9.38%
epoth: 0, iter_num: 1500, loss: 0.5084, 10.05%
epoth: 0, iter_num: 1600, loss: 0.3515, 10.72%
epoth: 0, iter_num: 1700, loss: 0.6481, 11.39%
epoth: 0, iter_num: 1800, loss: 0.2248, 12.06%
epoth: 0, iter_num: 1900, loss: 0.2088, 12.73%
epoth: 0, iter_num: 2000, loss: 0.4435, 13.40%
start validating...


  0%|          | 0/551 [00:00<?, ?it/s]

Accuracy: 0.6326
Average testing loss: 0.7390
-------------------------------
epoth: 0, iter_num: 2100, loss: 0.5067, 14.07%
epoth: 0, iter_num: 2200, loss: 0.3252, 14.74%
epoth: 0, iter_num: 2300, loss: 0.2887, 15.41%
epoth: 0, iter_num: 2400, loss: 0.4167, 16.08%
epoth: 0, iter_num: 2500, loss: 0.3514, 16.75%
epoth: 0, iter_num: 2600, loss: 0.6134, 17.42%
epoth: 0, iter_num: 2700, loss: 0.3206, 18.09%
epoth: 0, iter_num: 2800, loss: 0.5279, 18.76%
epoth: 0, iter_num: 2900, loss: 0.4058, 19.43%
epoth: 0, iter_num: 3000, loss: 0.6448, 20.10%
epoth: 0, iter_num: 3100, loss: 0.2499, 20.77%
epoth: 0, iter_num: 3200, loss: 0.3799, 21.44%
epoth: 0, iter_num: 3300, loss: 0.3192, 22.11%
epoth: 0, iter_num: 3400, loss: 0.5416, 22.78%
epoth: 0, iter_num: 3500, loss: 0.5221, 23.45%
epoth: 0, iter_num: 3600, loss: 0.5171, 24.12%
epoth: 0, iter_num: 3700, loss: 0.5124, 24.79%
epoth: 0, iter_num: 3800, loss: 0.2734, 25.46%
epoth: 0, iter_num: 3900, loss: 0.2892, 26.13%
epoth: 0, iter_num: 4000, los

...

Accuracy: 0.6615
Average testing loss: 0.6912
-------------------------------
epoth: 1, iter_num: 10100, loss: 0.2770, 67.68%
epoth: 1, iter_num: 10200, loss: 0.6129, 68.35%
epoth: 1, iter_num: 10300, loss: 0.3995, 69.02%
epoth: 1, iter_num: 10400, loss: 0.3941, 69.69%
epoth: 1, iter_num: 10500, loss: 0.1320, 70.36%
epoth: 1, iter_num: 10600, loss: 0.5397, 71.03%
epoth: 1, iter_num: 10700, loss: 0.3295, 71.70%
epoth: 1, iter_num: 10800, loss: 0.3763, 72.37%
epoth: 1, iter_num: 10900, loss: 0.5937, 73.04%
epoth: 1, iter_num: 11000, loss: 0.3672, 73.71%
epoth: 1, iter_num: 11100, loss: 0.3007, 74.38%
epoth: 1, iter_num: 11200, loss: 0.5849, 75.05%
epoth: 1, iter_num: 11300, loss: 0.2548, 75.72%
epoth: 1, iter_num: 11400, loss: 0.4279, 76.39%
epoth: 1, iter_num: 11500, loss: 0.4964, 77.06%
epoth: 1, iter_num: 11600, loss: 0.4257, 77.73%
epoth: 1, iter_num: 11700, loss: 0.7598, 78.40%
epoth: 1, iter_num: 11800, loss: 0.3319, 79.07%
epoth: 1, iter_num: 11900, loss: 0.2025, 79.74%
epoth: 1, 

  0%|          | 0/551 [00:00<?, ?it/s]

Accuracy: 0.6623
Average testing loss: 0.6905
-------------------------------
epoth: 1, iter_num: 12100, loss: 0.1745, 81.08%
epoth: 1, iter_num: 12200, loss: 0.3585, 81.75%
epoth: 1, iter_num: 12300, loss: 0.3248, 82.42%
epoth: 1, iter_num: 12400, loss: 0.2921, 83.09%
epoth: 1, iter_num: 12500, loss: 0.3009, 83.76%
epoth: 1, iter_num: 12600, loss: 0.3824, 84.43%
epoth: 1, iter_num: 12700, loss: 0.4865, 85.10%
epoth: 1, iter_num: 12800, loss: 0.3900, 85.77%
epoth: 1, iter_num: 12900, loss: 0.3494, 86.44%
epoth: 1, iter_num: 13000, loss: 0.7520, 87.11%
epoth: 1, iter_num: 13100, loss: 0.4261, 87.78%
epoth: 1, iter_num: 13200, loss: 0.4549, 88.45%
epoth: 1, iter_num: 13300, loss: 0.6242, 89.12%
epoth: 1, iter_num: 13400, loss: 0.5382, 89.79%
epoth: 1, iter_num: 13500, loss: 0.2922, 90.46%
epoth: 1, iter_num: 13600, loss: 0.4296, 91.13%
epoth: 1, iter_num: 13700, loss: 0.5860, 91.80%
epoth: 1, iter_num: 13800, loss: 0.2475, 92.47%
epoth: 1, iter_num: 13900, loss: 0.5761, 93.14%
epoth: 1, 

  0%|          | 0/551 [00:00<?, ?it/s]

Accuracy: 0.6623
Average testing loss: 0.6905
-------------------------------
epoth: 1, iter_num: 14100, loss: 0.3787, 94.49%
epoth: 1, iter_num: 14200, loss: 0.5444, 95.16%
epoth: 1, iter_num: 14300, loss: 0.5031, 95.83%
epoth: 1, iter_num: 14400, loss: 0.2669, 96.50%
epoth: 1, iter_num: 14500, loss: 0.2130, 97.17%
epoth: 1, iter_num: 14600, loss: 0.4135, 97.84%
epoth: 1, iter_num: 14700, loss: 0.4612, 98.51%
epoth: 1, iter_num: 14800, loss: 0.1833, 99.18%
epoth: 1, iter_num: 14900, loss: 0.3405, 99.85%
Epoch: 1, Average training loss: 0.4236


In [39]:
model = BertForNextSentencePrediction.from_pretrained('G:\\deep_learning\\Coggle\\202301\\models\\bert\\best_model')
model.to(device)
def test():
    print("start testing...")
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    for batch in tqdm(test_dataloader):
        with torch.no_grad():
            # 正常传播
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
        loss = outputs[0]
        logits = outputs[1]

        total_eval_loss += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        
    avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
    print("Accuracy: %.4f" % (avg_val_accuracy))
    print("Average testing loss: %.4f"%(total_eval_loss/len(test_dataloader)))
    print("-------------------------------")
    return avg_val_accuracy

test()

start testing...


  0%|          | 0/782 [00:00<?, ?it/s]

Accuracy: 0.6649
Average testing loss: 0.6602
-------------------------------


0.6648817135549873